In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


from tensorflow.keras import callbacks, optimizers
from yolov4.tf import SaveWeightsCallback, YOLOv4
import time

import sys
sys.path.append('..')
from datasets.datasets_utils import DatasetsUtils

# Convert annotations to YOLOv4 format

In [ ]:
data_utils = DatasetsUtils()

# Directories and prefixes

datasets_prefix = "../datasets/"
vehicles_cropped_dir = f"{datasets_prefix}vehicles_cropped"


# Generating annotation files compatible with YOLOv4 for all datasets.
all_anno_files = []


for i, dataset in enumerate(data_utils.datasets):
    anno_file_path = data_utils.generate_lp_det_data(i, vehicles_cropped_dir, prefix=datasets_prefix)
    all_anno_files.append(anno_file_path)
    print(i, dataset, "Done")

In [ ]:
# all_anno_files

all_splitted_anno = []
for anno_file in all_anno_files:
    
    if anno_file is not None:
        train, val, test = data_utils.split_dataset(anno_file, train=70, val=20)  # The rest will be test set.
        all_splitted_anno.append((train, val, test))
        
        
    
    print(anno_file)

# Adding fixed split dataset.
all_splitted_anno.append((f"{vehicles_cropped_dir}/ufpr_alpr/training/ufpr_alpr_yolo_anno.txt",
                          f"{vehicles_cropped_dir}/ufpr_alpr/validation/ufpr_alpr_yolo_anno.txt",
                          f"{vehicles_cropped_dir}/ufpr_alpr/testing/ufpr_alpr_yolo_anno.txt"))

In [ ]:
all_splitted_anno

In [ ]:
# Combining all train, val, and test sets of all datasets.
train_annos = []
val_annos = []
test_annos = []

for splitted_anno in all_splitted_anno:
    train_annos.append(splitted_anno[0])
    val_annos.append(splitted_anno[1])
    test_annos.append(splitted_anno[2])


train_anno_file = data_utils.combine_anno_files(train_annos, datasets_prefix, "all_train.txt")
val_anno_file = data_utils.combine_anno_files(val_annos, datasets_prefix, "all_val.txt")
test_anno_file = data_utils.combine_anno_files(test_annos, datasets_prefix, "all_test.txt")
    

In [ ]:
train_anno_file
val_anno_file

# Testing YOLOv4 annotation format 

In [1]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


from tensorflow.keras import callbacks, optimizers
from yolov4.tf import SaveWeightsCallback, YOLOv4
import time

import sys
sys.path.append('..')
from datasets.datasets_utils import DatasetsUtils


data_utils = DatasetsUtils()

# imgs_prefix = "test/dataset/val2017/"
# class_names_file = "test/dataset/coco.names"
# anno_file = "test/dataset/val2017.txt"

imgs_prefix = ""
class_names_file = "../datasets/class.names"
anno_file = "../datasets/all_train.txt"

# data_utils.check_yolov4_annos(imgs_prefix, class_names_file, anno_file)

# Training 

In [ ]:
len(yolo.classes)

In [2]:
datasets_prefix = "../datasets/"


yolo = YOLOv4(tiny=True)
# yolo.classes = f"{datasets_prefix}class.names"
yolo.classes = class_names_file

yolo.input_size = 608
yolo.batch_size = 1

yolo.make_model(activation1="relu")
yolo.load_weights("weights/yolov4-tiny.weights",
                  weights_type="yolo")


train_data_set = yolo.load_dataset(
    anno_file,
    image_path_prefix=imgs_prefix,
    label_smoothing=0.05
)
val_data_set = yolo.load_dataset(
    anno_file,
    image_path_prefix=imgs_prefix,
    training=False
)


# train_data_set = yolo.load_dataset(f"{datasets_prefix}all_train.txt",
#                                   image_path_prefix="",
#                                   label_smoothing=0.05)

# val_data_set = yolo.load_dataset(f"{datasets_prefix}all_val.txt",
#                                   image_path_prefix="",
#                                   training=False)

ValueError: Model and weights file do not match.

In [ ]:
YOLOv4?

In [ ]:
yolo.model.summary()

In [ ]:
epochs = 2
lr = 1e-4

optimizer = optimizers.Adam(learning_rate=lr)
yolo.compile(optimizer=optimizer, loss_iou_type="ciou")

def lr_scheduler(epoch):
    if epoch < int(epochs * 0.5):
        return lr
    if epoch < int(epochs * 0.8):
        return lr * 0.5
    if epoch < int(epochs * 0.9):
        return lr * 0.1
    return lr * 0.01

_callbacks = [
    callbacks.LearningRateScheduler(lr_scheduler),
    callbacks.TerminateOnNaN(),
    callbacks.TensorBoard(
        log_dir="test/logs",
    ),
    SaveWeightsCallback(
        yolo=yolo, dir_path="test/saved_weights",
        weights_type="yolo", epoch_per_save=100
    ),
]

In [ ]:
his = yolo.fit(
    train_data_set,
    epochs=epochs,
    callbacks=_callbacks,
    validation_data=val_data_set,
    validation_steps=50,
    validation_freq=5,
    steps_per_epoch=100,
)